In [1]:
import json
data = sc.textFile('hdfs://master:54310/stemmed'). \
    map(json.loads)

In [2]:
data.flatMap(lambda p: set(p['Text'])).count()

27882459

In [3]:
low_freq_words = data.flatMap(lambda p: set(p['Text'])). \
    map(lambda w: (w, 1)). \
    reduceByKey(lambda a, b: a+b). \
    filter(lambda x: x[1] < 5). \
    collectAsMap()

In [4]:
def exclude_low_freq_words(words: list):
    return [w for w in words if w not in low_freq_words]

In [5]:
data.take(1)[0].keys()

dict_keys(['Tags', 'Hubs', 'Text', 'Comments', 'Id', 'CodeComments'])

In [17]:
all_hubs = data.flatMap(lambda x: x['Hubs']).distinct()
all_tags = data.flatMap(lambda x: x['Tags']).distinct()

all_hubs.count(), all_tags.count()

(1563, 124421)

In [6]:
hub_bl = {'лог компании', 'рная дыра', 'пиарюсь'}

In [42]:
tag_count = data.flatMap(lambda x: x['Tags']). \
    map(lambda x: x.lower()). \
    countByValue()
hub_count = data.flatMap(lambda x: x['Hubs']). \
    map(lambda x: x.lower()). \
    filter(lambda h: all(h_bl not in h for h_bl in hub_bl)). \
    countByValue()

In [18]:
from matplotlib import pyplot as plt

In [33]:
tag_counts_array = []
hub_counts_array = []
counts = range(0, 5000, 10)

for c in range(0, 5000, 10):
    tag_counts_array.append(len([v for v in tag_count.values() if v >= c]))
    hub_counts_array.append(len([v for v in hub_count.values() if v >= c]))

plt.plot(counts, tag_counts_array, 'b', hold=True)
plt.plot(counts, hub_counts_array, 'r')
plt.show()

In [43]:
all_labels = dict(tag_count)
all_labels.update(hub_count)

In [37]:
all_labels_counts_array = []
counts = range(0, 5000, 10)

for c in range(0, 5000, 10):
    all_labels_counts_array.append(len([v for v in all_labels.values() if v >= c]))

plt.plot(counts, all_labels_counts_array, 'b', hold=True)
plt.show()

In [44]:
from collections import Counter

In [45]:
label_counts = Counter(all_labels)

In [49]:
most_common = dict(label_counts.most_common(500)).keys()

In [60]:
label_counts = data.map(lambda d: (d['Tags'], d['Hubs'])). \
    map(lambda x: ([t.lower() for t in x[0]],
                   [h.lower() for h in x[1]])). \
    map(lambda x: (x[0], [h for h in x[1] if all(h_bl not in h for h_bl in hub_bl)])). \
    map(lambda x: set(x[0] + x[1])). \
    map(lambda x: len(x)). \
    collect()

In [64]:
plt.hist(label_counts, bins=60)
plt.show()

In [8]:
new_labels = data.map(lambda d: (d['Tags'], d['Hubs'])). \
    map(lambda x: ([t.lower() for t in x[0]],
                   [h.lower() for h in x[1]])). \
    map(lambda x: (x[0], [h for h in x[1] if all(h_bl not in h for h_bl in hub_bl)])). \
    map(lambda x: set(x[0] + x[1]))

In [12]:
'кол-во всех документов', new_labels.count()

('кол-во всех документов', 127434)

In [9]:
more_than_3 = new_labels.filter(lambda x: len(x) > 3)
'кол-во всех документов', more_than_3.count()

('кол-во всех документов', 85981)

In [14]:
all_labels_from_more_than_3_rdd = more_than_3.flatMap(lambda x: x).distinct()
'уникальных лейблов в документах, где больше 3', all_labels_from_more_than_3_rdd.count()

('уникальных лейблов в документах, где больше 3', 97645)

In [10]:
all_labels_counts = more_than_3.flatMap(lambda x: x). \
    map(lambda x: (x,1)). \
    reduceByKey(lambda a,b: a+b). \
    sortBy(lambda x: x[1], ascending=False)

In [32]:
all_labels_count_list = all_labels_counts.map(lambda x: x[1]).collect()

In [33]:
plt.hist(all_labels_count_list, bins=30)
plt.show()

In [34]:
all_labels_counts.take(10)

[('веб-разработка', 4538),
 ('информационная безопасность', 3846),
 ('программирование', 3725),
 ('google', 3220),
 ('android', 3055),
 ('javascript', 2978),
 ('настройка linux', 2530),
 ('linux', 2442),
 ('microsoft', 2431),
 ('разработка', 2419)]

In [35]:
all_labels_counts.filter(lambda x: x[1] > 10).count()

5763

In [38]:
top_500 = all_labels_counts.map(lambda x: x[0]).take(500)
top_500[:5]

['веб-разработка',
 'информационная безопасность',
 'программирование',
 'google',
 'android']

In [50]:
top_500_set = set(top_500)
def delete_not_top_500_labels(labels: set):
    return [l for l in labels if l in top_500_set]

In [51]:
lengths_after_del = new_labels.map(lambda x: delete_not_top_500_labels(x)). \
    map(lambda x: len(x)).collect()

In [52]:
plt.hist(lengths_after_del, bins=30)
plt.show()

In [53]:
new_labels.map(lambda x: delete_not_top_500_labels(x)). \
    map(lambda x: len(x)). \
    countByValue()

defaultdict(<class 'int'>, {0: 15986, 1: 41645, 2: 35134, 3: 19796, 4: 8742, 5: 3568, 6: 1362, 7: 566, 8: 250, 9: 94, 10: 82, 11: 42, 12: 13, 13: 9, 14: 49, 15: 30, 16: 41, 17: 18, 18: 6, 20: 1})

In [54]:
new_new = new_labels.map(lambda x: delete_not_top_500_labels(x)). \
    filter(lambda x: 2 < len(x) < 10)
new_new.count()

34378

In [55]:
all_labels_counts.zipWithIndex().filter(lambda x: x[1] < 500).map(lambda x: x[0]).sortBy(lambda x: x[1]).take(10)

[('go', 117),
 ('тест', 117),
 ('статистика в it', 117),
 ('трафик', 118),
 ('ux', 119),
 ('управление', 119),
 ('svn', 119),
 ('школа', 120),
 ('пароли', 121),
 ('системы управления версиями', 121)]

In [ ]:
new_labels.map(lambda x: delete_not_top_500_labels(x)). \
    filter(lambda x: len(x) > 8).collect()

### ТОП 100

In [11]:
top_1000 = all_labels_counts.map(lambda x: x[0]).take(1000)
top_1000[:5]

['веб-разработка',
 'информационная безопасность',
 'программирование',
 'google',
 'android']

In [12]:
top_1000_set = set(top_1000)
def delete_not_top_1000_labels(labels: set):
    return [l for l in labels if l in top_1000_set]

In [59]:
lengths_after_del_2 = new_labels.map(lambda x: delete_not_top_1000_labels(x)). \
    map(lambda x: len(x)).collect()

In [60]:
plt.hist(lengths_after_del_2, bins=30)
plt.show()

In [61]:
new_labels.map(lambda x: delete_not_top_1000_labels(x)). \
    map(lambda x: len(x)). \
    countByValue()

defaultdict(<class 'int'>, {0: 10802, 1: 33549, 2: 35508, 3: 24348, 4: 12705, 5: 5641, 6: 2498, 7: 1085, 8: 592, 9: 258, 10: 157, 11: 88, 12: 29, 13: 15, 14: 42, 15: 41, 16: 34, 17: 21, 18: 15, 19: 3, 20: 1, 21: 1, 22: 1})

In [62]:
all_labels_counts.zipWithIndex().filter(lambda x: x[1] < 1000).map(lambda x: x[0]).sortBy(lambda x: x[1]).take(10)

[('extensions', 61),
 ('coworking', 61),
 ('ocr', 61),
 ('управление персоналом', 61),
 ('accessibility', 61),
 ('lumia', 61),
 ('ip', 61),
 ('календарь', 61),
 ('вопросы', 61),
 ('ностальгия', 61)]

In [65]:
new_labels.map(lambda x: delete_not_top_1000_labels(x)). \
    map(lambda x: len(x)). \
    filter(lambda x: x>2).count()

47575

In [70]:
new_labels.map(lambda x: delete_not_top_1000_labels(x)). \
    flatMap(lambda x: x). \
    map(lambda x: (x,1)). \
    reduceByKey(lambda a,b: a+b). \
    map(lambda x: x[1]). \
    filter(lambda x: x < 50). \
    countByValue()

defaultdict(<class 'int'>, {})

In [13]:
top_1000 = all_labels_counts.map(lambda x: x[0]).take(1000)
top_1000_set = set(top_1000)

In [14]:
def choose_labels(doc: dict):
    all_labels = doc['Tags'] + doc['Hubs']
    all_labels = set([l.lower() for l in all_labels])
    return [l for l in all_labels if l in top_1000_set]

In [15]:
def transform_data(doc: dict):
    return {
        'Id': doc['Id'],
        'Features': exclude_low_freq_words(doc['Text']),
        'Labels': choose_labels(doc)
    }

In [16]:
data.map(transform_data). \
    filter(lambda x: len(x['Labels']) > 2). \
    count()

47575

In [89]:
data.map(transform_data). \
    filter(lambda x: len(x['Labels']) > 2). \
    flatMap(lambda x: x['Features']). \
    distinct(). \
    count()

98621

In [17]:
data.map(transform_data). \
    filter(lambda x: len(x['Labels']) > 2). \
    map(lambda x: json.dumps(x)). \
    repartition(4). \
    saveAsTextFile('hdfs://master:54310/clean')